In [16]:
# USING ENSEMBLE KNOWLEDGE DISTILLATION TO TRAIN THE STUDENT MODEL

import subprocess, sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import timm
except ImportError:
    install("timm")
    import timm

# IMPORTS
import os
import time
import random
import numpy as np
import pandas as pd
import cv2
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
from torchvision import models

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix


# CONFIG
DATASET_DIR = Path("/kaggle/input/rice-disease-dataset/Rice_Leaf_AUG")

TEACHER_PATHS = {
    "resnet": "/kaggle/input/teacher-weights/pytorch/default/1/best_model_resnet18.pth",
    "mobilenet": "/kaggle/input/teacher-weights/pytorch/default/1/best_model_mobilenet.pth",
    "densenet": "/kaggle/input/teacher-weights/pytorch/default/1/best_model_densenet.pth",
}

NUM_CLASSES = 6
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 25
LR = 3e-4
ALPHA = 0.8
TEMPERATURE = 4.0

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("=" * 60)
print(f"ENSEMBLE KD TRAINING ON {DEVICE}")
print("=" * 60)

# ============================================================
# SEED
# ============================================================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


# DATASET PREPARATION
filepaths, labels = [], []

for cls in sorted(os.listdir(DATASET_DIR)):
    cls_dir = DATASET_DIR / cls
    if cls_dir.is_dir():
        for img in os.listdir(cls_dir):
            filepaths.append(str(cls_dir / img))
            labels.append(cls)

df = pd.DataFrame({"filepath": filepaths, "label": labels})

encoder = LabelEncoder()
df["label_encoded"] = encoder.fit_transform(df["label"])
class_names = encoder.classes_

print("Classes:", class_names)

train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)


# AUGMENTATIONS
def get_train_transforms():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.CLAHE(clip_limit=4.0, p=0.5),
        A.CoarseDropout(
            max_holes=8,
            max_height=20,
            max_width=20,
            fill_value=0,
            p=0.5
        ),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),
        ToTensorV2()
    ])

def get_val_transforms():
    return A.Compose([
        A.Resize(IMG_SIZE, IMG_SIZE),
        A.Normalize(
            mean=(0.485, 0.456, 0.406),
            std=(0.229, 0.224, 0.225)
        ),
        ToTensorV2()
    ])

class RiceDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(row["filepath"])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = row["label_encoded"]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return img, torch.tensor(label, dtype=torch.long)

train_loader = DataLoader(
    RiceDataset(train_df, get_train_transforms()),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    RiceDataset(val_df, get_val_transforms()),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


# TEACHER MODELS 
class RiceResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(weights=None)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

class RiceMobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.mobilenet_v2(weights=None)
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(1280, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

class RiceDenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.densenet121(weights=None)

        num_ftrs = self.backbone.classifier.in_features
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, NUM_CLASSES)
        )

    def forward(self, x):
        return self.backbone(x)

def clean_state_dict(state_dict):
    new_state = {}
    for k, v in state_dict.items():
        if k.startswith("module."):
            k = k.replace("module.", "")
        if k.startswith("model."):
            k = k.replace("model.", "")
        new_state[k] = v
    return new_state



# LOAD TEACHERS
def load_teacher(model, path, name):
    print(f"Loading {name} teacher...")
    state = torch.load(path, map_location=DEVICE)
    state = clean_state_dict(state)
    if isinstance(model, RiceDenseNet):
        model.backbone.load_state_dict(state, strict=True)
    else:
        model.model.load_state_dict(state, strict=True)

    model.to(DEVICE)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False

    print(f"{name} loaded successfully")
    return model


teachers = [
    load_teacher(RiceResNet(), TEACHER_PATHS["resnet"], "ResNet18"),
    load_teacher(RiceMobileNet(), TEACHER_PATHS["mobilenet"], "MobileNetV2"),
    load_teacher(RiceDenseNet(), TEACHER_PATHS["densenet"], "DenseNet121"),
]

print(f"Loaded {len(teachers)} teachers successfully")


# STUDENT MODEL
class StudentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model(
            "mobilenetv3_small_050",
            pretrained=True
        )
        num_ftrs = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.Hardswish(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.Hardswish(),
            nn.Dropout(0.1),
            nn.Linear(128, NUM_CLASSES)
        )

    def forward(self, x):
        return self.model(x)

student = StudentModel().to(DEVICE)


# KD LOSS
class EnsembleKDLoss(nn.Module):
    def __init__(self, alpha=0.8, T=4.0):
        super().__init__()
        self.alpha = alpha
        self.T = T
        self.ce = nn.CrossEntropyLoss(label_smoothing=0.1)

    def forward(self, s_logits, targets, t_logits):
        ce_loss = self.ce(s_logits, targets)
        kd_loss = F.kl_div(
            F.log_softmax(s_logits / self.T, dim=1),
            F.softmax(t_logits / self.T, dim=1),
            reduction="batchmean"
        ) * (self.T ** 2)

        return self.alpha * kd_loss + (1 - self.alpha) * ce_loss

criterion = EnsembleKDLoss(ALPHA, TEMPERATURE)
optimizer = optim.AdamW(student.parameters(), lr=LR, weight_decay=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
scaler = torch.cuda.amp.GradScaler()


# TRAIN / EVAL
def train_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(loader, leave=False):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        with torch.no_grad():
            t_logits = torch.mean(
                torch.stack([t(imgs) for t in teachers]),
                dim=0
            )

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            s_logits = model(imgs)
            loss = criterion(s_logits, labels, t_logits)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * imgs.size(0)
        preds = s_logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            logits = model(imgs)
            preds = logits.argmax(1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return correct / total, all_labels, all_preds


# TRAIN LOOP
best_acc = 0.0
print("\nStarting Knowledge Distillation...\n")

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(student, train_loader)
    val_acc, y_true, y_pred = evaluate(student, val_loader)
    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{EPOCHS}] "
        f"Loss: {train_loss:.4f} "
        f"Train Acc: {train_acc:.4f} "
        f"Val Acc: {val_acc:.4f}"
    )

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(student.state_dict(), "best_student.pth")
        print(">> Best Student Saved")


# FINAL REPORT
print("=" * 40)
print(f"BEST VALIDATION ACCURACY: {best_acc:.4f}")

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:\n", cm)


ENSEMBLE KD TRAINING ON cuda
Classes: ['Bacterial Leaf Blight' 'Brown Spot' 'Healthy Rice Leaf' 'Leaf Blast'
 'Leaf scald' 'Sheath Blight']


/tmp/ipykernel_55/3590033457.py:117: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(


Loading ResNet18 teacher...
ResNet18 loaded successfully
Loading MobileNetV2 teacher...
MobileNetV2 loaded successfully
Loading DenseNet121 teacher...
DenseNet121 loaded successfully
Loaded 3 teachers successfully


/tmp/ipykernel_55/3590033457.py:322: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Starting Knowledge Distillation...



...

Epoch [21/25] Loss: 0.2196 Train Acc: 0.9549 Val Acc: 0.9556


Epoch [22/25] Loss: 0.2161 Train Acc: 0.9582 Val Acc: 0.9608
>> Best Student Saved


Epoch [23/25] Loss: 0.2135 Train Acc: 0.9576 Val Acc: 0.9582


Epoch [24/25] Loss: 0.2136 Train Acc: 0.9611 Val Acc: 0.9608


Epoch [25/25] Loss: 0.2153 Train Acc: 0.9549 Val Acc: 0.9569
BEST VALIDATION ACCURACY: 0.9608

Confusion Matrix:
 [[124   0   0   0   3   0]
 [  2 119   0   4   3   1]
 [  0   0 131   0   0   0]
 [  4   3   0 118   1   1]
 [  0   1   0   3 121   1]
 [  3   1   1   1   0 120]]


In [2]:
pip install torch torchvision onnx onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 106.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [36]:
# DOING QAT for QUANTIZATION AND CONVERT TO INT8 ONNX FORMAT

import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import torch.quantization.quantize_fx as quantize_fx
import onnx
import onnxruntime
import numpy as np
import timm
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

# 1. CONFIGURATION
MODEL_INPUT_PATH = "/kaggle/working/best_student.pth"
ONNX_OUTPUT_PATH = "/kaggle/working/student_int8_qat_optimized.onnx"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# High-Performance Hyperparameters
TOTAL_EPOCHS = 50     
FREEZE_EPOCH = 10      
START_LR = 1e-4         
MIN_LR = 1e-6           

print(f"Running Optimized QAT on: {DEVICE}")
print(f"Strategy: 1-{FREEZE_EPOCH} Learn Ranges | {FREEZE_EPOCH+1}-{TOTAL_EPOCHS} Fine-tune Weights")


# 2. DEFINE MODEL
class StudentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model("mobilenetv3_small_050", pretrained=False)
        num_ftrs = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.Hardswish(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.Hardswish(),
            nn.Dropout(0.1),
            nn.Linear(128, 6)
        )
    def forward(self, x):
        return self.model(x)


# 3. PREPARE FOR QAT
print("\n[1/6] Loading & Fusing Model...")
model = StudentModel().to(DEVICE)
model.load_state_dict(torch.load(MODEL_INPUT_PATH, map_location=DEVICE))
qconfig_dict = {"": torch.quantization.get_default_qat_qconfig('qnnpack')}
model.train() 

example_input = torch.randn(1, 3, 224, 224).to(DEVICE)
model_prepared = quantize_fx.prepare_qat_fx(model, qconfig_dict, example_inputs=example_input)
print("      Model prepared.")


# 4. OPTIMIZED TRAINING LOOP
print(f"\n[2/6] Starting QAT Process...")

optimizer = optim.Adam(model_prepared.parameters(), lr=START_LR)
scheduler = CosineAnnealingLR(optimizer, T_max=TOTAL_EPOCHS, eta_min=MIN_LR)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

best_acc = 0.0
best_model_weights = copy.deepcopy(model_prepared.state_dict())

for epoch in range(TOTAL_EPOCHS):
    model_prepared.train()
    
    # === FREEZE OBSERVERS ===
    if epoch == FREEZE_EPOCH:
        print(f"\n      >>> STABILIZATION PHASE: Freezing Observers & BN Stats <<<")
        model_prepared.apply(torch.quantization.disable_observer)
        model_prepared.apply(torch.nn.intrinsic.qat.freeze_bn_stats)
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{TOTAL_EPOCHS}")
    for imgs, labels in pbar:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model_prepared(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # Stats
        running_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        pbar.set_postfix({'loss': loss.item(), 'acc': f"{correct/total:.2%}"})

    scheduler.step()
    
    # Save best
    epoch_acc = correct / total
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_weights = copy.deepcopy(model_prepared.state_dict())

print(f"\n      Training Complete. Best Accuracy: {best_acc*100:.2f}%")
model_prepared.load_state_dict(best_model_weights)


# 5. CONVERSION & EXPORT
print("\n[3/6] Moving to CPU for Conversion...")
model_prepared.to("cpu")
model_prepared.eval()

print("      Converting to INT8...")
quantized_model = quantize_fx.convert_fx(model_prepared)

print(f"\n[4/6] Exporting to ONNX: {ONNX_OUTPUT_PATH}")
dummy_input_cpu = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    quantized_model,
    dummy_input_cpu,
    ONNX_OUTPUT_PATH,
    input_names=['input'],
    output_names=['output'],
    opset_version=13,
    do_constant_folding=True,
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} 
)


# 6. FINAL VALIDATION
print("\n[5/6] Final Verification on Validation Set...")
session = onnxruntime.InferenceSession(ONNX_OUTPUT_PATH, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

correct = 0
total = 0

for imgs, labels in tqdm(val_loader, desc="Testing ONNX"):
    img_numpy = imgs.numpy()
    ort_outs = session.run([output_name], {input_name: img_numpy})
    preds = np.argmax(ort_outs[0], axis=1)
    correct += (preds == labels.numpy()).sum()
    total += labels.size(0)

final_acc = correct / total

print("\n" + "="*40)
print(f"FINAL OPTIMIZED INT8 ACCURACY: {final_acc*100:.2f}%")
print("="*40)

if final_acc > 0.90:
    print("✅ MISSION ACCOMPLISHED: 90%+ INT8 Model Created.")

Running Optimized QAT on: cuda
Strategy: 1-10 Learn Ranges | 11-50 Fine-tune Weights

[1/6] Loading & Fusing Model...


/tmp/ipykernel_55/2152863518.py:63: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  model_prepared = quantize_fx.prepare_qat_fx(model, qconfig_dict, example_inputs=example_input)
/usr/local/lib/python3.12/dist-packages/torch/ao/quantization/quantize_fx.py:146: FutureWarning: Passing a QConfig dictionary to prepare is deprecated and will not be supported in a future version

      Model prepared.

[2/6] Starting QAT Process...


Epoch 10/50: 100%|██████████| 96/96 [00:18<00:00,  5.31it/s, loss=0.777, acc=91.02%]



      >>> STABILIZATION PHASE: Freezing Observers & BN Stats <<<


...


      Training Complete. Best Accuracy: 97.29%

[3/6] Moving to CPU for Conversion...
      Converting to INT8...

[4/6] Exporting to ONNX: /kaggle/working/student_int8_qat_optimized.onnx


/tmp/ipykernel_55/2152863518.py:139: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/usr/local/lib/python3.12/dist-packages/torch/onnx/symbolic_helper.py:1460: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'batch_norm' is set to train=True. Exporting with train=True.
  warnings.warn(



[5/6] Final Verification on Validation Set...


Testing ONNX: 100%|██████████| 24/24 [00:07<00:00,  3.33it/s]


FINAL OPTIMIZED INT8 ACCURACY: 91.25%
✅ MISSION ACCOMPLISHED: 90%+ INT8 Model Created.


In [5]:
# MODEL ANALYSIS

import os
import time
import numpy as np
import onnx
import onnxruntime as ort
from onnx import numpy_helper

# 1. SETUP
ONNX_MODEL_PATH = "/kaggle/input/int8-qat-optimized-student/pytorch/default/1/student_int8_qat_optimized.onnx"
INPUT_SHAPE = (1, 3, 224, 224)

if not os.path.exists(ONNX_MODEL_PATH):
    raise FileNotFoundError(f"Model not found at {ONNX_MODEL_PATH}")

print(f"Analyzing Model: {ONNX_MODEL_PATH}...\n")


# 2. FILE SIZE
size_bytes = os.path.getsize(ONNX_MODEL_PATH)
size_mb = size_bytes / (1024 * 1024)


# 3. NUMBER OF PARAMETERS
model_onnx = onnx.load(ONNX_MODEL_PATH)
total_params = 0
for initializer in model_onnx.graph.initializer:
    dims = initializer.dims
    if dims:
        total_params += np.prod(dims)

# 4. INFERENCE TIME (LATENCY)
session = ort.InferenceSession(ONNX_MODEL_PATH, providers=['CPUExecutionProvider'])
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

dummy_input = np.random.randn(*INPUT_SHAPE).astype(np.float32)


print("Warming up inference engine...")
for _ in range(50):
    session.run([output_name], {input_name: dummy_input})

# Measurement Loop
iterations = 1000
print(f"Running {iterations} iterations for timing...")

latencies = []
for _ in range(iterations):
    start_time = time.time()
    session.run([output_name], {input_name: dummy_input})
    end_time = time.time()
    latencies.append((end_time - start_time) * 1000)

avg_latency = np.mean(latencies)
fps = 1000 / avg_latency

# 5. FINAL REPORT
print("\n" + "="*50)
print(f"📊 FINAL INT8 MODEL METRICS")
print("="*50)
print(f"💾 Model File Size:       {size_mb:.2f} MB")
print(f"🧠 Total Parameters:      {total_params:,}")
print(f"⏱️ Avg Inference Time:    {avg_latency:.4f} ms per image (CPU)")
print(f"🚀 Throughput:            {fps:.2f} FPS")
print("="*50)

# Comparison Context
print("\nComparison Context:")
print(f"- Float32 Size Estimate:  {size_mb * 4:.2f} MB (approx)")
print(f"- Size Reduction:         {((size_mb*4 - size_mb)/(size_mb*4))*100:.0f}% smaller")

Analyzing Model: /kaggle/input/int8-qat-optimized-student/pytorch/default/1/student_int8_qat_optimized.onnx...

Warming up inference engine...
Running 1000 iterations for timing...

📊 FINAL INT8 MODEL METRICS
💾 Model File Size:       1.10 MB
🧠 Total Parameters:      13,812
⏱️ Avg Inference Time:    4.4274 ms per image (CPU)
🚀 Throughput:            225.87 FPS

Comparison Context:
- Float32 Size Estimate:  4.40 MB (approx)
- Size Reduction:         75% smaller
